In [ ]:
import sys
sys.path.append('../src/')
from xpcs import *
from sims import *
from autocorrelations import *
import cv2
from scipy.special import erfinv
import pyopencl as cl

%matplotlib notebook

# Find 2Th Position From Detector

In [ ]:
det_width = x1.shape[0]*x_pixel_size # Microns
angular_span = (det_width/D)*180/np.pi #Angular span of whole detector
dThdpix = angular_span/x1.shape[0] #Degrees per pixel, roughly

pixXcen = np.arange(-y1.shape[0]//2,y1.shape[0]//2)

thetas = np.flip(pixXcen*dThdpix+ttheta) #Thetas for th given detector position

# Plot as a function of Q

In [ ]:
x = Qx[0,:]
# Data
poptth, pcovth = curve_fit(gaussian, x, y1, p0=[np.max(y1), x[np.argmax(np.flip(y1))] , 2, np.min(y1[np.nonzero(y1)])])

# Create a figure and a set of subplots
fig, axs = plt.subplots(2, 1, figsize=(8, 4))  # 2 rows, 1 column

# Plot on the first axis
axs[0].plot(x,y1)
axs[0].plot(x, gaussian(x, *poptth), label='mu={}\nsigma={}'.format(poptth[1],poptth[2]), color='red')
axs[0].set_title('Q_x')
axs[0].legend()
axs[1].plot(q_to_tth(x,wavelength),y1)

# Sample Information

In [ ]:
hkl = [1,2,1]

a,b,c = 7.07114, 7.07114, 7.07114

alpha, beta, gamma = 90.000000, 90.000000, 90.000000

# Create Mask

In [ ]:
f=0.5 #Block size in standard deviations
block_shape = [int(np.rint(hSig*f)),int(np.rint(hSig*f))]
num_blocks = [25,17] #vertical, horizotanl. Odd numbers only
blocks,coords = create_square_mask(det,block_shape,num_blocks)

## My Way

In [ ]:

coords = np.array(coords)
iCen = np.shape(coords)[0]//2 #Get center ROI index

coordCen = coords[iCen] #Get center ROI coordinate

dPixelXY = (coords-coordCen)*np.array([x_pixel_size,y_pixel_size]) #ROI distance coordinates in pixels

dPixDist = np.sqrt(np.sum(dPixelXY**2,axis=1)) #ROIs distances In microns

dTheta = (dPixDist/(D*10**(-3)))*180/np.pi
print(f'ΔTh:{dTheta}')

delQ = (4*np.pi/wavelength)*np.sin(0.5*dTheta*np.pi/180)
print(f'Q_cen:{delQ}')


## Vanyas Way

In [ ]:
#Calculate momentum transfer in x and y directions
qx,qy = reciprocal_space_map(wavelength,ttheta,X0,Y0,0.075,D,detDims)

#Create a meshgrid to map to the detector shpae
Qx,Qy = np.meshgrid(qx,qy)

#print(f'Qx:{Qx}')
#print(f'Qy:{Qy}')

#Set to same ROI as the detector
Qx = Qx[roi[0]:roi[1],roi[2]:roi[3]]
Qy = Qy[roi[0]:roi[1],roi[2]:roi[3]]

Qx0 = Qx[coordCen[1],coordCen[0]]
Qy0 = Qy[coordCen[1],coordCen[0]]

#dQ = np.sqrt((Qx-Qx0)**2+(Qy-Qy0)**2)

dQ = np.sqrt((Qx-Qx0)**2+(Qy-Qy0)**2)[coords[:,1],coords[:,0]]

dQ = np.around(dQ,5)

print(f'ΔQ:{(delQ-dQ)/delQ}')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook


# Create a figure and axis
fig, ax = plt.subplots()
# Display the image
ax.imshow(np.sum(det[0:5,:,:],axis=0)+1,cmap='nipy_spectral',norm=LogNorm(vmin=1, vmax=35))

# Draw the rectangles
for (x, y) in coords:
    # Create a rectangle patch at the given coordinates with the width and height of the block
    # Rectangle sets the origin in the bottom-left corner of the rectangle. Need to shift to center 
    rect = Rectangle((x-block_shape[1]//2, y-block_shape[0]//2), block_shape[1], block_shape[0], linewidth=1, edgecolor='r', facecolor='none')
    ax.add_patch(rect)

# Show the plot with the ROIs
plt.show()